# Data Prep

In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score
from sklearn.metrics import f1_score, matthews_corrcoef

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset

import psutil
import time
import logging
from datetime import datetime
import random
import os
import shutil
import itertools

import ray
from ray import tune
from ray.tune.schedulers import ASHAScheduler
from ray.air import session
from ray.tune.integration.keras import TuneReportCallback
from ray.tune.search.optuna import OptunaSearch
from ray.tune.tuner import Tuner, TuneConfig
from ray.train import RunConfig
from ray.tune import Trainable

import helper

# Initialize Ray
ray.init(ignore_reinit_error=True)

# Check if MPS (Apple Silicon) or CUDA (Nvidia) GPU is available
if torch.backends.mps.is_available():
    device = torch.device('mps')
elif torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

print(f"Using device: {device}")

2024-08-11 15:25:23,271	INFO worker.py:1772 -- Started a local Ray instance. View the dashboard at 127.0.0.1:8266 


Using device: mps


## Existing Data Files Check 

In [2]:
def clear_folder(folder_path):
    for item in os.listdir(folder_path):
        item_path = os.path.join(folder_path, item)
        if item == '.gitignore':
            continue
        if os.path.isfile(item_path):
            os.remove(item_path)
        elif os.path.isdir(item_path):
            shutil.rmtree(item_path)

clear_folder("ray_results")
clear_folder("outputs")
clear_folder("results")

## Simple FNN

In [3]:
class Classic_FFN(nn.Module):
    def __init__(self, num_classes, hidden_layers=[30, 15], dropout_rate=0.5, activation='relu'):
        super(Classic_FFN, self).__init__()

        self.activation = activation

        self.shared_layers = nn.ModuleList()
        self.conv_layers = nn.ModuleList()

        self.conv_layers.append(nn.Conv1d(in_channels=1, out_channels=32, kernel_size=3, padding=1))
        self.conv_layers.append(nn.BatchNorm1d(32))
        self.conv_layers.append(nn.MaxPool1d(kernel_size=2))
        self.conv_layers.append(nn.Conv1d(in_channels=32, out_channels=64, kernel_size=3, padding=1))
        self.conv_layers.append(nn.BatchNorm1d(64))
        self.conv_layers.append(nn.MaxPool1d(kernel_size=2))

        input_dim = 64 * (30 // 4)

        for hidden_layer in hidden_layers:
            self.shared_layers.append(nn.Linear(input_dim, hidden_layer))
            self.shared_layers.append(nn.Dropout(dropout_rate))
            input_dim = hidden_layer

        self.output_layer = nn.Linear(input_dim, num_classes)

    def forward(self, x):
        # Convolutional layers
        x = x.unsqueeze(1) # Add channel dimension
        for conv in self.conv_layers:
            if isinstance(conv, nn.Conv1d):
                x = conv(x)
                x = self.apply_activation(x)
            elif isinstance(conv, nn.BatchNorm1d):
                x = conv(x)
            elif isinstance(conv, nn.MaxPool1d):
                x = conv(x)

        x = x.view(x.size(0), -1) # Flatten the output for the fully connected layers

        for layer in self.shared_layers:
            if isinstance(layer, nn.Linear):
                x = layer(x)
                x = self.apply_activation(x)
            elif isinstance(layer, nn.Dropout):
                x = layer(x)
            
        x = self.output_layer(x)
        return x

    def apply_activation(self, x):
        if self.activation == 'relu':
            return F.relu(x)
        elif self.activation == 'tanh':
            return torch.tanh(x)
        elif self.activation == 'sigmoid':
            return torch.sigmoid(x)
        else:
            raise ValueError(f'Invalid activation: {self.activation}')

In [4]:
def create_model(num_classes, hidden_layers=[30,15], dropout_rate=0.5, activation='relu'):
    model = Classic_FFN(num_classes=num_classes, hidden_layers=hidden_layers, dropout_rate=dropout_rate, activation=activation)
    model.to(device)  # Move the model to the selected device
    return model

def train(model, X_train, y_train, num_epochs, batch_size, learning_rate):
    # Convert data to PyTorch tensors and move to the selected device
    X_train_tensor = torch.tensor(X_train, dtype=torch.float32).to(device)
    y_train_tensor = torch.tensor(np.argmax(y_train, axis=1), dtype=torch.long).to(device)  # Convert one-hot to class indices

    # Create a DataLoader for the training data
    train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
    train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
    
    # Define loss function and optimizer
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    
    # Training loop
    for epoch in range(num_epochs):
        model.train()  # Set the model to training mode
        for X_batch, y_batch in train_loader:
            optimizer.zero_grad()  # Clear the gradients
            outputs = model(X_batch)  # Forward pass
            loss = criterion(outputs, y_batch)  # Calculate loss
            loss.backward()  # Backward pass
            optimizer.step()  # Update weights
    
    return model

def evaluate(model, X_test, y_test):
    # Convert data to PyTorch tensors and move to the selected device
    X_test_tensor = torch.tensor(X_test, dtype=torch.float32).to(device)
    y_test_tensor = torch.tensor(y_test, dtype=torch.float32).to(device)
    
    # Set model to evaluation mode
    model.eval()
    
    # Disable gradient calculation for inference
    with torch.no_grad():
        outputs = model(X_test_tensor)  # Forward pass
        y_pred_classes = torch.argmax(outputs, dim=1).cpu().numpy()  # Get predicted classes and move to CPU
        y_test_classes = torch.argmax(y_test_tensor, dim=1).cpu().numpy()  # Get true classes and move to CPU
        
        # Calculate accuracy
        accuracy = (y_pred_classes == y_test_classes).mean()
        
        # Calculate F1 scores and MCC
        f1_macro = f1_score(y_test_classes, y_pred_classes, average='macro')
        f1_micro = f1_score(y_test_classes, y_pred_classes, average='micro')
        mcc = matthews_corrcoef(y_test_classes, y_pred_classes)
    
    return accuracy, f1_macro, f1_micro, mcc

In [5]:
def evaluate_model_on_dataset_one_split(config, split, split_index):
    test_set = split[split_index]
    indices = [0, 1, 2, 3, 4]
    indices.remove(split_index)
    train_splits = [split[i] for i in indices]
    train_set = pd.concat(train_splits, axis=0)

    classes = train_set['track'].unique()
    num_classes = len(classes)
    #print("number of classes: " + str(num_classes))

    one_hot_columns = train_set['track'].unique()
    one_hot = pd.get_dummies(train_set['track'])
    train_set = train_set.drop('track', axis=1)
    train_set = train_set.join(one_hot).astype(float)

    one_hot_columns = test_set['track'].unique()
    one_hot = pd.get_dummies(test_set['track'])
    test_set = test_set.drop('track', axis=1)
    test_set = test_set.join(one_hot).astype(float)

    X_train = train_set.drop(columns=one_hot_columns).values.reshape(-1, 30)
    y_train = train_set[one_hot_columns].values.reshape(-1, num_classes)
    X_test = test_set.drop(columns=one_hot_columns).values.reshape(-1, 30)
    y_test = test_set[one_hot_columns].values.reshape(-1, num_classes)

    model = create_model(num_classes, activation=config['activation'], hidden_layers=config['hidden_layers'], dropout_rate=config['dropout_rate'])

    # Train the model and collect performance data
    model= train(model=model, X_train=X_train, y_train=y_train, num_epochs=config['epochs'], batch_size=config['batch_size'], learning_rate=config['learning_rate'])
    # Evaluate the model and collect performance data
    accuracy, f1_macro, f1_micro, mcc = evaluate(model=model, X_test=X_test, y_test=y_test)


    return accuracy, f1_macro, f1_micro, mcc, num_classes

## 5-Fold-Cross Validation

In [6]:
df_data_spike_1_split = pd.DataFrame()
df_data_spike_full_split = pd.DataFrame()

data_spike_exec_1_split_dict = dict()
data_spike_exec_full_split_dict = dict()


best_params_list_getting = []

def custom_trial_dirname(trial):
    return f"trial_{trial.trial_id}"

In [7]:
def train_and_evaluate(config, splits, splits_name):
    
    global data_spike_exec_1_split_dict
    global data_spike_exec_full_split_dict
    global df_data_spike_1_split
    global results_dir
    
    min_accuracy = 1
    max_accuracy = 0
    avg_accuracy = 0
    min_f1_macro = 1
    max_f1_macro = 0
    avg_f1_macro = 0
    min_f1_micro = 1
    max_f1_micro = 0
    avg_f1_micro = 0
    min_mcc = 1
    max_mcc = -1
    avg_mcc = 0

    num_classes = 0
    
    accuracies = []
    f1_macro_scores = []
    f1_micro_scores = []
    mcc_scores = []

    session_id_for_df = session.get_trial_id()
    
    for i in range(5):
        accuracy, f1_macro, f1_micro, mcc, num_classes = evaluate_model_on_dataset_one_split(config, splits, i)
        accuracies.append(accuracy)
        f1_macro_scores.append(f1_macro)
        f1_micro_scores.append(f1_micro)
        mcc_scores.append(mcc)
        
        avg_accuracy += accuracy
        avg_f1_macro += f1_macro
        avg_f1_micro += f1_micro
        min_accuracy = min(min_accuracy, accuracy)
        max_accuracy = max(max_accuracy, accuracy)
        min_f1_macro = min(min_f1_macro, f1_macro)
        max_f1_macro = max(max_f1_macro, f1_macro)
        min_f1_micro = min(min_f1_micro, f1_micro)
        max_f1_micro = max(max_f1_micro, f1_micro)
        avg_mcc += mcc
        min_mcc = min(min_mcc, mcc)
        max_mcc = max(max_mcc, mcc)


        
        data_spike_exec_1_split_dict[splits_name + "_" + str(i+1) + "_" + session_id_for_df] = {
            "number of classes": num_classes,
            "accuracy": accuracy,
            "macro f1": f1_macro,
            "micro_f1": f1_micro,
            "mcc": mcc,
            "config": str(config)
        }
        
       
    temp_df = pd.DataFrame.from_dict(data_spike_exec_1_split_dict, orient='index')
    df_data_spike_1_split = pd.concat([df_data_spike_1_split, temp_df], axis=0)

        # df_data_spike_1_split = pd.concat([df_data_spike_1_split, pd.DataFrame.from_dict(data_spike_exec_1_split_dict)], axis=1)

        
    avg_accuracy /= 5
    avg_f1_macro /= 5
    avg_f1_micro /= 5
    avg_mcc /= 5
    
    data_spike_exec_full_split_dict[splits_name + "_" + session_id_for_df] = {
        "number of classes": num_classes,
        "min_accuracy": min_accuracy,
        "max_accuracy": max_accuracy,
        "mean_accuracy": np.mean(accuracies),
        "min_f1_macro": min_f1_macro,
        "max_f1_macro": max_f1_macro,
        "mean_f1_macro": np.mean(f1_macro_scores),
        "min_f1_micro": min_f1_micro,
        "max_f1_micro": max_f1_micro,
        "mean_f1_micro": np.mean(f1_micro_scores),
        "min_mcc": min_mcc,
        "max_mcc": max_mcc,
        "mean_mcc": np.mean(mcc_scores),
        "std_accuracy": np.std(accuracies),
        "std_f1_macro": np.std(f1_macro_scores),
        "std_f1_micro": np.std(f1_micro_scores),
        "std_mcc": np.std(mcc_scores),
        "config": str(config)
    }

    # print(f"Mean accuracy of the model {splits_name + '_' + session_id_for_df}: {avg_accuracy}")

    df_data_spike_full_split = pd.DataFrame.from_dict(data_spike_exec_full_split_dict, orient='index')

    # Load existing data from file if it exists and append new data
    full_split_path = os.path.join(helper.results_dir, f'output_full_{splits_name}.pkl')
    if os.path.exists(full_split_path):
        df_existing_full = pd.read_pickle(full_split_path)
        df_data_spike_full_split = pd.concat([df_existing_full, df_data_spike_full_split], axis=0)
    else:
        print(f"No existing full split data found at {full_split_path}, creating new file.")
    
    # Save the updated full split data to file
    df_data_spike_full_split.to_pickle(full_split_path)

    # Save the 1 split data using the full path
    split_path = os.path.join(helper.results_dir, f'output_{splits_name}.pkl')
    if os.path.exists(split_path):
        df_existing_1_spike = pd.read_pickle(split_path)
        df_data_spike_1_split = pd.concat([df_existing_1_spike, df_data_spike_1_split], axis=0)
    else:
        print(f"No existing 1 split data found at {split_path}, creating new file")

    # Save the updated 1 split data to file
    df_data_spike_1_split.to_pickle(split_path)

    time.sleep(5)
    
    session.report({
        "min_accuracy": min_accuracy,
        "max_accuracy": max_accuracy,
        "mean_accuracy": np.mean(accuracies),
        "min_f1_macro": min_f1_macro,
        "max_f1_macro": max_f1_macro,
        "mean_f1_macro": np.mean(f1_macro_scores),
        "min_f1_micro": min_f1_micro,
        "max_f1_micro": max_f1_micro,
        "mean_f1_micro": np.mean(f1_micro_scores)
    })

In [8]:
def generate_hidden_layers_config(min_layers=2, max_layers=5, min_nodes=30, max_nodes=90, step=5):
    possible_layers = []
    for num_layers in range(min_layers, max_layers + 1):
        layer_configurations = list(itertools.product(range(min_nodes, max_nodes + 1, step), repeat=num_layers))
        possible_layers.extend(layer_configurations)
    return possible_layers

def five_fold_cross_validation(splits, splits_name):

    global best_params_list_getting

    hidden_layers_options = generate_hidden_layers_config()

    config = {
        "activation": tune.choice(["relu", "tanh", "sigmoid"]),
        "learning_rate": tune.loguniform(1e-4, 1e-2),
        "batch_size": tune.choice([32, 64, 128]),
        "hidden_layers": tune.choice(hidden_layers_options),
        "epochs": tune.choice([10, 20, 30, 40, 50]),
        "dropout_rate": tune.uniform(0.2, 0.5)
    }
    
    scheduler = ASHAScheduler(
        metric="mean_accuracy",
        mode="max",
        max_t=10,
        grace_period=1,
        reduction_factor=2
    )
    
    search_alg = OptunaSearch(metric="mean_accuracy", mode="max")
    
    analysis = tune.run(
        tune.with_parameters(train_and_evaluate, splits=splits, splits_name=splits_name),
        resources_per_trial={"cpu": 10, "gpu": 0, "accelerator_type:RTX": 0},
        config=config,
        scheduler=scheduler,
        search_alg=search_alg,
        num_samples=32,
        verbose=1,
        storage_path=helper.ray_results_dir,
        trial_dirname_creator=custom_trial_dirname
    )

    best_config_data_ray_tune = analysis.get_best_config(metric="mean_accuracy", mode="max")
    print("Best hyperparameters found were: ", best_config_data_ray_tune)
    best_params_list_getting.append(best_config_data_ray_tune)
    
    return analysis

In [9]:
analysis = five_fold_cross_validation(helper.a2_splits, "A2")

(train_and_evaluate pid=25644) No existing full split data found at /Users/bhanuprasanna/Documents/Uniklinik-Koln/MTL/spike-sorting-multi-task/results/output_full_A2.pkl, creating new file.
(train_and_evaluate pid=25644) No existing 1 split data found at /Users/bhanuprasanna/Documents/Uniklinik-Koln/MTL/spike-sorting-multi-task/results/output_A2.pkl, creating new file


2024-08-11 15:25:58,854	INFO tensorboardx.py:308 -- Removed the following hyperparameter values when logging to tensorboard: {'hidden_layers': (70, 85, 45, 80, 40)}
2024-08-11 15:26:26,217	INFO tensorboardx.py:308 -- Removed the following hyperparameter values when logging to tensorboard: {'hidden_layers': (30, 50, 55, 80, 60)}
2024-08-11 15:26:49,419	INFO tensorboardx.py:308 -- Removed the following hyperparameter values when logging to tensorboard: {'hidden_layers': (60, 80, 45, 70, 85)}
2024-08-11 15:27:01,469	INFO tensorboardx.py:308 -- Removed the following hyperparameter values when logging to tensorboard: {'hidden_layers': (40, 90, 65, 30, 60)}
2024-08-11 15:27:41,488	INFO tensorboardx.py:308 -- Removed the following hyperparameter values when logging to tensorboard: {'hidden_layers': (85, 70, 80, 70)}
2024-08-11 15:28:16,332	INFO tensorboardx.py:308 -- Removed the following hyperparameter values when logging to tensorboard: {'hidden_layers': (70, 85, 65, 90, 50)}
2024-08-11 15:

Best hyperparameters found were:  {'activation': 'relu', 'learning_rate': 0.00018153627532969336, 'batch_size': 64, 'hidden_layers': (90, 90, 65, 55, 40), 'epochs': 30, 'dropout_rate': 0.4941268198413076}


In [10]:
analysis = five_fold_cross_validation(helper.a3_splits, "A3")

(train_and_evaluate pid=27542) No existing full split data found at /Users/bhanuprasanna/Documents/Uniklinik-Koln/MTL/spike-sorting-multi-task/results/output_full_A3.pkl, creating new file.
(train_and_evaluate pid=27542) No existing 1 split data found at /Users/bhanuprasanna/Documents/Uniklinik-Koln/MTL/spike-sorting-multi-task/results/output_A3.pkl, creating new file


2024-08-11 15:38:29,305	INFO tensorboardx.py:308 -- Removed the following hyperparameter values when logging to tensorboard: {'hidden_layers': (65, 50, 50, 50, 60)}
2024-08-11 15:38:47,887	INFO tensorboardx.py:308 -- Removed the following hyperparameter values when logging to tensorboard: {'hidden_layers': (75, 75, 85, 90, 80)}
2024-08-11 15:39:05,176	INFO tensorboardx.py:308 -- Removed the following hyperparameter values when logging to tensorboard: {'hidden_layers': (85, 75, 35, 35, 50)}
2024-08-11 15:39:32,369	INFO tensorboardx.py:308 -- Removed the following hyperparameter values when logging to tensorboard: {'hidden_layers': (70, 90, 55, 35)}
2024-08-11 15:40:02,539	INFO tensorboardx.py:308 -- Removed the following hyperparameter values when logging to tensorboard: {'hidden_layers': (70, 90, 65, 60, 45)}
2024-08-11 15:40:15,734	INFO tensorboardx.py:308 -- Removed the following hyperparameter values when logging to tensorboard: {'hidden_layers': (55, 55, 85, 45, 60)}
2024-08-11 15:

Best hyperparameters found were:  {'activation': 'relu', 'learning_rate': 0.00255156012554144, 'batch_size': 32, 'hidden_layers': (85, 75, 40, 80, 75), 'epochs': 10, 'dropout_rate': 0.4796967270268306}


In [11]:
analysis = five_fold_cross_validation(helper.a4_splits, "A4")

(train_and_evaluate pid=28566) No existing full split data found at /Users/bhanuprasanna/Documents/Uniklinik-Koln/MTL/spike-sorting-multi-task/results/output_full_A4.pkl, creating new file.
(train_and_evaluate pid=28566) No existing 1 split data found at /Users/bhanuprasanna/Documents/Uniklinik-Koln/MTL/spike-sorting-multi-task/results/output_A4.pkl, creating new file


2024-08-11 15:47:59,432	INFO tensorboardx.py:308 -- Removed the following hyperparameter values when logging to tensorboard: {'hidden_layers': (85, 70, 65, 45, 70)}
2024-08-11 15:48:41,502	INFO tensorboardx.py:308 -- Removed the following hyperparameter values when logging to tensorboard: {'hidden_layers': (85, 50, 40, 50)}
2024-08-11 15:49:02,794	INFO tensorboardx.py:308 -- Removed the following hyperparameter values when logging to tensorboard: {'hidden_layers': (80, 75, 65, 55, 50)}
2024-08-11 15:49:23,966	INFO tensorboardx.py:308 -- Removed the following hyperparameter values when logging to tensorboard: {'hidden_layers': (35, 55, 90, 40, 30)}
2024-08-11 15:50:38,129	INFO tensorboardx.py:308 -- Removed the following hyperparameter values when logging to tensorboard: {'hidden_layers': (85, 35, 85, 45, 45)}
2024-08-11 15:52:37,564	INFO tensorboardx.py:308 -- Removed the following hyperparameter values when logging to tensorboard: {'hidden_layers': (30, 75, 35, 80, 65)}
2024-08-11 15:

Best hyperparameters found were:  {'activation': 'relu', 'learning_rate': 0.003991246077136659, 'batch_size': 128, 'hidden_layers': (35, 90, 50, 40, 45), 'epochs': 40, 'dropout_rate': 0.2412349299577167}


In [12]:
analysis = five_fold_cross_validation(helper.a12_splits, "A12")

(train_and_evaluate pid=31163) No existing full split data found at /Users/bhanuprasanna/Documents/Uniklinik-Koln/MTL/spike-sorting-multi-task/results/output_full_A12.pkl, creating new file.
(train_and_evaluate pid=31163) No existing 1 split data found at /Users/bhanuprasanna/Documents/Uniklinik-Koln/MTL/spike-sorting-multi-task/results/output_A12.pkl, creating new file


2024-08-11 16:13:03,160	INFO tensorboardx.py:308 -- Removed the following hyperparameter values when logging to tensorboard: {'hidden_layers': (70, 70, 40, 80, 50)}
2024-08-11 16:14:01,730	INFO tensorboardx.py:308 -- Removed the following hyperparameter values when logging to tensorboard: {'hidden_layers': (70, 65, 55, 45, 70)}
2024-08-11 16:14:19,339	INFO tensorboardx.py:308 -- Removed the following hyperparameter values when logging to tensorboard: {'hidden_layers': (50, 75, 60, 30, 30)}
2024-08-11 16:15:56,614	INFO tensorboardx.py:308 -- Removed the following hyperparameter values when logging to tensorboard: {'hidden_layers': (75, 90, 85, 60, 45)}
2024-08-11 16:16:27,513	INFO tensorboardx.py:308 -- Removed the following hyperparameter values when logging to tensorboard: {'hidden_layers': (60, 85, 85, 65, 90)}
2024-08-11 16:17:00,904	INFO tensorboardx.py:308 -- Removed the following hyperparameter values when logging to tensorboard: {'hidden_layers': (55, 30, 55, 30, 30)}
2024-08-11

Best hyperparameters found were:  {'activation': 'tanh', 'learning_rate': 0.0009752032860232802, 'batch_size': 32, 'hidden_layers': (75, 90, 40, 30, 90), 'epochs': 30, 'dropout_rate': 0.36000954648695677}


In [13]:
analysis = five_fold_cross_validation(helper.a21_splits, "A21")

(train_and_evaluate pid=34469) No existing full split data found at /Users/bhanuprasanna/Documents/Uniklinik-Koln/MTL/spike-sorting-multi-task/results/output_full_A21.pkl, creating new file.
(train_and_evaluate pid=34469) No existing 1 split data found at /Users/bhanuprasanna/Documents/Uniklinik-Koln/MTL/spike-sorting-multi-task/results/output_A21.pkl, creating new file


2024-08-11 16:44:59,789	INFO tensorboardx.py:308 -- Removed the following hyperparameter values when logging to tensorboard: {'hidden_layers': (40, 70, 40, 80, 70)}
2024-08-11 16:45:26,176	INFO tensorboardx.py:308 -- Removed the following hyperparameter values when logging to tensorboard: {'hidden_layers': (40, 90, 60, 65, 90)}
2024-08-11 16:45:46,096	INFO tensorboardx.py:308 -- Removed the following hyperparameter values when logging to tensorboard: {'hidden_layers': (30, 75, 80, 75, 90)}
2024-08-11 16:46:26,286	INFO tensorboardx.py:308 -- Removed the following hyperparameter values when logging to tensorboard: {'hidden_layers': (85, 85, 80, 80, 45)}
2024-08-11 16:47:00,590	INFO tensorboardx.py:308 -- Removed the following hyperparameter values when logging to tensorboard: {'hidden_layers': (50, 60, 30, 60, 80)}
2024-08-11 16:47:15,875	INFO tensorboardx.py:308 -- Removed the following hyperparameter values when logging to tensorboard: {'hidden_layers': (65, 65, 30, 80)}
2024-08-11 16:

Best hyperparameters found were:  {'activation': 'relu', 'learning_rate': 0.0004103576794828613, 'batch_size': 32, 'hidden_layers': (75, 50, 75, 65, 70), 'epochs': 50, 'dropout_rate': 0.45227663901390397}


In [14]:
df_dict_A2 = pd.read_pickle("results/output_A2.pkl")
df_dict_A3 = pd.read_pickle("results/output_A3.pkl")
df_dict_A4 = pd.read_pickle("results/output_A4.pkl")
df_dict_A12 = pd.read_pickle("results/output_A12.pkl")
df_dict_A21 = pd.read_pickle("results/output_A21.pkl")

df_dict = pd.concat([df_dict_A2, df_dict_A3, df_dict_A4, df_dict_A12, df_dict_A21])

df_dict = df_dict.loc[:,~df_dict.columns.duplicated()].copy()
df_dict = df_dict.reset_index().rename(columns={'index': 'name'})
df_dict = df_dict.drop_duplicates()

df_dict.to_csv('outputs/Classic_FFN_one_split_metrics_system.csv')

In [15]:
df_dict_full_A2 = pd.read_pickle("results/output_full_A2.pkl")
df_dict_full_A3 = pd.read_pickle("results/output_full_A3.pkl")
df_dict_full_A4 = pd.read_pickle("results/output_full_A4.pkl")
df_dict_full_A12 = pd.read_pickle("results/output_full_A12.pkl")
df_dict_full_A21 = pd.read_pickle("results/output_full_A21.pkl")

df_full_dict = pd.concat([df_dict_full_A2, df_dict_full_A3, df_dict_full_A4, df_dict_full_A12, df_dict_full_A21])

df_full_dict = df_full_dict.loc[:, ~df_full_dict.columns.duplicated()].copy()
df_full_dict = df_full_dict.reset_index().rename(columns={'index': 'name'})
df_full_dict = df_full_dict.drop_duplicates()

df_full_dict.to_csv("outputs/Classic_FFN_full_split_metrics_system.csv")

In [16]:
df_dict_full_A2.to_csv("outputs/Classic_FFN_full_split_metrics_A2.csv")
df_dict_full_A3.to_csv("outputs/Classic_FFN_full_split_metrics_A3.csv")
df_dict_full_A4.to_csv("outputs/Classic_FFN_full_split_metrics_A4.csv")
df_dict_full_A12.to_csv("outputs/Classic_FFN_full_split_metrics_A12.csv")
df_dict_full_A21.to_csv("outputs/Classic_FFN_full_split_metrics_A21.csv")

In [39]:
# print the configs and results for the hyperparemters wiht the highest mean accuracy
# read df_full_dict and print columns with configs in best_params_list_getting
df_full_dict = pd.read_csv("outputs/Classic_FFN_full_split_metrics_system.csv")
df_full_dict.drop(columns=['Unnamed: 0'], inplace=True)

config_acc_dict = {}
for good_param in best_params_list_getting:
    # get the row witht the highest mean mean accuracy
    mean_accuracy = df_full_dict.loc[df_full_dict['config'] == str(good_param)]['mean_accuracy'].mean()
    config_acc_dict[str(good_param)] = mean_accuracy
    print(f"Mean accuracy for config {good_param} is {mean_accuracy}")

# get the best config
best_config = max(config_acc_dict, key=config_acc_dict.get)
print(f"\n\nThe best config is {best_config} with a mean accuracy of {config_acc_dict[best_config]}")

Mean accuracy for config {'activation': 'relu', 'learning_rate': 0.00018153627532969336, 'batch_size': 64, 'hidden_layers': (90, 90, 65, 55, 40), 'epochs': 30, 'dropout_rate': 0.4941268198413076} is 0.949705780546902
Mean accuracy for config {'activation': 'relu', 'learning_rate': 0.00255156012554144, 'batch_size': 32, 'hidden_layers': (85, 75, 40, 80, 75), 'epochs': 10, 'dropout_rate': 0.4796967270268306} is 0.924927536231884
Mean accuracy for config {'activation': 'relu', 'learning_rate': 0.003991246077136659, 'batch_size': 128, 'hidden_layers': (35, 90, 50, 40, 45), 'epochs': 40, 'dropout_rate': 0.2412349299577167} is 0.7116299195308766
Mean accuracy for config {'activation': 'tanh', 'learning_rate': 0.0009752032860232802, 'batch_size': 32, 'hidden_layers': (75, 90, 40, 30, 90), 'epochs': 30, 'dropout_rate': 0.36000954648695677} is 0.3182980546518322
Mean accuracy for config {'activation': 'relu', 'learning_rate': 0.0004103576794828613, 'batch_size': 32, 'hidden_layers': (75, 50, 75

In [41]:
best_performers = df_full_dict.loc[df_full_dict.groupby(df_full_dict['name'].str.split('_').str[0])['mean_accuracy'].idxmax()]
best_performers = best_performers.sort_index().reset_index(drop=True)

best_performers

,name,number of classes,min_accuracy,max_accuracy,mean_accuracy,min_f1_macro,max_f1_macro,mean_f1_macro,min_f1_micro,max_f1_micro,mean_f1_micro,min_mcc,max_mcc,mean_mcc,std_accuracy,std_f1_macro,std_f1_micro,std_mcc,config
0,A2_527500c4,2,0.907407,0.971963,0.949706,0.907121,0.971953,0.949639,0.907407,0.971963,0.949706,0.819892,0.945436,0.901048,0.022256,0.022362,0.022256,0.042945,"{'activation': 'relu', 'learning_rate': 0.0001..."
1,A3_96349dd3,2,0.898551,0.956522,0.924928,0.898551,0.956522,0.924858,0.898551,0.956522,0.924928,0.797479,0.916397,0.851265,0.026286,0.026287,0.026286,0.052860,"{'activation': 'relu', 'learning_rate': 0.0025..."
2,A4_eb1bde45,3,0.647208,0.753181,0.711630,0.638725,0.752376,0.708532,0.647208,0.753181,0.711630,0.490295,0.631115,0.572734,0.039143,0.041674,0.039143,0.052306,"{'activation': 'relu', 'learning_rate': 0.0039..."
3,A12_54dc37f5,6,0.266839,0.351421,0.318298,0.245337,0.325706,0.302332,0.266839,0.351421,0.318298,0.121942,0.225464,0.184049,0.028498,0.029590,0.028498,0.034524,"{'activation': 'tanh', 'learning_rate': 0.0009..."
4,A21_8d426873,2,0.704762,0.761905,0.743324,0.693647,0.761125,0.726623,0.704762,0.761905,0.743324,0.398184,0.525728,0.468759,0.020220,0.025642,0.020220,0.047783,"{'activation': 'relu', 'learning_rate': 0.0004..."
